In [1]:
import os, json
import numpy as np
import pandas as pd
from dLoader import DataLoader

# Get Tickers from Database

In [6]:
directory = os.path.join(os.getcwd(), 'Database')
# Search Param
min_dollar = 0
max_dollar = 10
min_length = 250 * 6
today = pd.Timestamp.now()
# Placeholder
tickers = []
files = os.listdir(directory)
if 'keeper.json' in files:
    path = os.path.join(directory, 'keeper.json')
    db = pd.read_json(path, orient='index')
    # Get pricing
    mask = (db['Latest Price'].between(min_dollar, max_dollar)) & (db['Length in Database'] > min_length)
    ndb = db.loc[mask].copy()
    need_update = (today.day - pd.to_datetime(ndb['Last Updated']).dt.day) > 90
    if need_update.sum() > 0:
        jFile = json.load(open(path, 'r'))
        for ticker in ndb.loc[need_update].index.to_list():
            DataLoader(ticker).update_database()
            jFile[ticker]['Last Updated'] = today.strftime('%Y-%m-%d')
        # Write json file
        json.dump(jFile, open(path, 'w'), indent=4)
    tickers = db.loc[mask].index.to_list()
else:
    jFile = {}
    for file in files:
        name, ext = file.split('.')
        if ext == 'csv':
            df = DataLoader(name).data
            latest_price = df['Close'].iloc[-1]
            jFile[name] = {'Latest Price': latest_price,
                           'Length in Database': len(df),
                           'Last Updated': df.index[-1].strftime('%Y-%m-%d')}
            if len(df) > min_length and (min_dollar < latest_price < max_dollar):
                tickers.append(name)
    # Write json file
    with open(os.path.join(directory, 'keeper.json'), 'w') as outfile:
        json.dump(jFile, outfile, indent=4)

print('Total tickers met parameters in Database: {}'.format(len(tickers)))

Total tickers met parameters in Database: 253


In [321]:
ticker = np.random.choice(tickers)
data = DataLoader(ticker).get_data('2021-01-01', '2021-12-31')

In [322]:
# ATR
atr_period = 5
hl = data['High'] - data['Low']
hc = (data['High'] - data['Close'].shift(1)).abs()
lc = (data['Low'] - data['Close'].shift(1)).abs()
data['TR'] = np.stack([hl, hc, lc], axis=1).max(1)
data['ATR'] = data['TR'].ewm(alpha=1/atr_period, min_periods=atr_period).mean()

In [323]:
# Supertrend
multiplier = 1
up = data.loc[:, ['High', 'Low']].mean(1) + (multiplier * data['ATR'])
down = data.loc[:, ['High', 'Low']].mean(1) - (multiplier * data['ATR'])

In [324]:
# Buy Sell Singal
g1 = data['High'].shift(-1) / data['Close'] - 1
g2 = data['High'].shift(-2).rolling(2).max() / data['Close'] - 1
g3 = data['High'].shift(-3).rolling(3).max() / data['Close'] - 1
g4 = data['High'].shift(-4).rolling(4).max() / data['Close'] - 1
g5 = data['High'].shift(-5).rolling(5).max() / data['Close'] - 1
supertrend = pd.DataFrame(np.stack([data['Close'].shift(-1), up, down, 
                                    g1, g2, g3, g4, g5], axis=1),
                          columns=['Close', 'Upperband', 'Lowerband', 
                                   'Gain1', 'Gain2', 'Gain3', 'Gain4', 'Gain5'],
                          index=data.index)
signal = {d: {'Previous_singal': np.nan,
              'Buy_at': 0,
              'Gain1': 0,
              'Gain2': 0,
              'Gain3': 0,
              'Gain4': 0,
              'Gain5': 0,
              'Signal': np.nan}
          for d in data.index}
for d, trend in supertrend.iterrows():
    c = trend['Close']
    u = trend.Upperband
    l = trend.Lowerband
    g1 = trend.Gain1
    g2 = trend.Gain2
    g3 = trend.Gain3
    g4 = trend.Gain4
    g5 = trend.Gain5
    if pd.isna(c) or pd.isna(u) or pd.isna(l) or pd.isna(g5):
        continue
    # Check Singal
    if c > u:
        signal[d]['Buy_at'] = c
        signal[d]['Gain1'] = g1
        signal[d]['Gain2'] = g2
        signal[d]['Gain3'] = g3
        signal[d]['Gain4'] = g4
        signal[d]['Gain5'] = g5
        signal[d]['Signal'] = 'Long'
    

In [325]:
df = pd.DataFrame.from_dict(signal, orient='index')

In [326]:
mask = df['Signal'] == 'Long'
print('Num of Trades', mask.sum())
print('Pct of Trades: {:.2f}%'.format(mask.sum() / len(data) * 100))
dic = {}
for pct in np.arange(0, .1, step=.01):
    dic[pct] = (df.loc[mask, ['Gain1', 'Gain2', 'Gain3', 'Gain4', 'Gain5']] > pct).mean(0)
pd.DataFrame.from_dict(dic, orient='index')

Num of Trades 16
Pct of Trades: 6.35%


,Gain1,Gain2,Gain3,Gain4,Gain5
0.00,1.0000,1.0000,1.0000,1.0000,1.0000
0.01,1.0000,1.0000,1.0000,1.0000,1.0000
0.02,1.0000,1.0000,1.0000,1.0000,1.0000
0.03,1.0000,1.0000,1.0000,1.0000,1.0000
0.04,1.0000,1.0000,1.0000,1.0000,1.0000
0.05,1.0000,1.0000,1.0000,1.0000,1.0000
0.06,0.9375,0.9375,0.9375,0.9375,0.9375
0.07,0.9375,0.9375,0.9375,0.9375,0.9375
0.08,0.8750,0.8750,0.8750,0.9375,0.9375
0.09,0.7500,0.8750,0.8750,0.9375,0.9375
